In [ ]:
# cell 1
import ee
ee.Authenticate()  # ยืนยันตัวตนกับ Google Earth Engine
project_id = "ee-sakda-451407"

ee.Initialize(project=project_id)  # เริ่มต้นใช้งานด้วย project ID ที่กำหนด


In [ ]:
# cell 2
!pip install folium

In [ ]:
# cell 3 folium
import folium   

def add_raster_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles = map_id_dict['tile_fetcher'].url_format,
        attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name = name,
        overlay = True,
        control = True
    ).add_to(self)

# เพิ่มฟังก์ชัน add_raster_layer เข้าไปใน folium.Map
folium.Map.add_raster_layer = add_raster_layer

def add_vector_layer(self, ee_feature_collection, vis_params, name):
    map_id_dict = ee.FeatureCollection(ee_feature_collection).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles = map_id_dict['tile_fetcher'].url_format,
        attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name = name,
        overlay = True,
        control = True
    ).add_to(self)


# เพิ่มฟังก์ชัน add_vector_layer เข้าไปใน folium.Map
folium.Map.add_vector_layer = add_vector_layer  


In [ ]:
# cell 4 geometry, feature, feature_collection

# สร้างจุด (Point) ที่พิกัดที่กำหนด
point = ee.Geometry.Point(98.9853, 18.7883)
# สร้างฟีเจอร์ (Feature) จากจุดที่สร้างขึ้น
feature = ee.Feature(point, {'name': 'My Point'})
# สร้างคอลเลกชันของฟีเจอร์ (FeatureCollection)
feature_collection = ee.FeatureCollection([feature])

print('Point:', point.getInfo())
print('Feature:', feature.getInfo())
print('FeatureCollection:', feature_collection.getInfo())


In [ ]:
# cell 5
# show vector data on map
fig1 = folium.Figure(height="300px")
m2 = folium.Map(location=[18.7883, 98.9853], zoom_start=14).add_to(fig1)

# กำหนดพารามิเตอร์การแสดงผลสำหรับฟีเจอร์คอลเลกชัน
vector_vis_params = {
    'color': 'red',
    'pointRadius': 10,
    'width': 2
}
# เพิ่มฟีเจอร์คอลเลกชันลงในแผนที่ด้วยพารามิเตอร์การแสดงผลที่กำหนด
m2.add_vector_layer(feature_collection, vector_vis_params, 'My Feature Collection')

folium.LayerControl().add_to(m2)

# แสดงแผนที่ผ่าน Figure เพื่อให้ render เป็น HTML
fig1


In [ ]:
# cell 6 - study area
# สร้างพื้นที่ศึกษา (Study Area) เป็นรูปสี่เหลี่ยมที่ครอบคลุมพื้นที่ที่สนใจ
bbox = ee.Geometry.Rectangle([98.95, 18.75, 99.02, 18.82])
print('Study Area:', bbox.getInfo())  

# cell 7 - show study area on map
fig2 = folium.Figure(height="300px")

m3 = folium.Map(location=[18.7883, 98.9853], zoom_start=12).add_to(fig2)
folium.GeoJson(bbox.getInfo()).add_to(m3)
folium.LayerControl().add_to(m3)

fig2

In [ ]:
# cell 7 image
# ดึงข้อมูลภาพจาก Sentinel-2 วันที่ 2024-01-09 ที่ครอบคลุมพื้นที่ที่กำหนด
s2_image = ee.Image('COPERNICUS/S2_SR_HARMONIZED/20250103T035049_20250103T040205_T47QMA') \
    .clip(bbox)  # ตัดภาพให้ตรงกับพื้นที่ที่กำหนด

print('Bands:', s2_image.bandNames().getInfo())
print('วันที่:', s2_image.date().format('YYYY-MM-dd').getInfo())

# กำหนดพารามิเตอร์การแสดงผลสำหรับภาพ Sentinel-2
s2_vis_params = {
    'bands': ['B4', 'B3', 'B2'],  # แสดงผลด้วยแถบสีแดง, เขียว, น้ำเงิน
    'min': 0,
    'max': 3000,
    'gamma': 1.4
}
# สร้างแผนที่ใหม่สำหรับแสดงภาพ Sentinel-2
fig3 = folium.Figure(height="300px")
m3 = folium.Map(location=[18.8, 98.95], zoom_start=12).add_to(fig3)
# เพิ่มภาพ Sentinel-2 ลงในแผนที่ด้วยพารามิเตอร์การแสดงผลที่กำหนด
m3.add_raster_layer(s2_image, s2_vis_params, 'Sentinel-2 Image')
folium.LayerControl().add_to(m3)    
# แสดงแผนที่ผ่าน Figure เพื่อให้ render เป็น HTML
fig3



In [ ]:
# cell 8 - ImageCollection

# โหลด ImageCollection ของ Sentinel-2 และกรองตามวันที่และพื้นที่
s2 = ee.ImageCollection('COPERNICUS/S2') \
    .filterDate('2025-01-01', '2025-12-31') \
    .filterBounds(bbox) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

# เลือกภาพที่มีเมฆน้อยที่สุด
img = s2.sort('CLOUDY_PIXEL_PERCENTAGE').median()
print('ภาพที่เลือก:', img.getInfo())
print('จำนวนภาพ:', s2.size().getInfo())
print('Bands:', img.bandNames().getInfo())
print('วันที่เริ่มต้น:', s2.first().date().format('YYYY-MM-dd').getInfo())
print('วันที่สิ้นสุด:', s2.sort('CLOUDY_PIXEL_PERCENTAGE').first().date().format('YYYY-MM-dd').getInfo())

# แสดงภาพบนแผนที่
fig4 = folium.Figure(height="300px")
m4 = folium.Map(location=[18.7883, 98.9853], zoom_start=12).add_to(fig4)

# แสดง True Color (B4, B3, B2)
m4.add_raster_layer(img, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'True Color')

# แสดง False Color (B8, B4, B3) - เน้นพืชพรรณ
m4.add_raster_layer(img, {'bands': ['B8', 'B4', 'B3'], 'min': 0, 'max': 3000}, 'False Color')

folium.LayerControl().add_to(m4)

fig4


In [ ]:
# cell 9 - คำนวณ NDVI และแสดงผล
fig5 = folium.Figure(height="300px")
m5 = folium.Map(location=[18.7883, 98.9853], zoom_start=12).add_to(fig5)

ndvi = img.normalizedDifference(['B8', 'B4']).rename('NDVI')
ndvi_palette = ['red', 'orange', 'yellow', 'lightgreen', 'green', 'darkgreen']
m5.add_raster_layer(ndvi, {'min': -0.2, 'max': 0.8, 'palette': ndvi_palette}, 'NDVI')

folium.LayerControl().add_to(m5)
fig5

In [ ]:
# cell 10 - band math
fig6 = folium.Figure(height="300px")
m6 = folium.Map(location=[18.7883, 98.9853], zoom_start=12).add_to(fig6)    
# คำนวณ NDWI (Normalized Difference Water Index) เพื่อแยกแยะน้ำ ด้วย สูตร (B3 - B8) / (B3 + B8) 

# เลือก band B3 (Green) และ B8 (NIR)
b3 = img.select('B3')
b8 = img.select('B8')

# คำนวณ NDWI แบบเขียนสูตรเอง ไม่ใช้ normalizedDifference()
ndwi = b3.subtract(b8).divide(b3.add(b8)).rename('NDWI')

# แสดงผล NDWI บนแผนที่ ด้วย palette จากน้ำตาล (พื้นดิน/ค่าต่ำ) ไปน้ำเงินเข้ม (น้ำ/ค่าสูง)
ndwi_palette = ['brown', 'yellow', 'cyan', 'blue', 'darkblue']
m6.add_raster_layer(ndwi, {'min': -0.5, 'max': 0.5, 'palette': ndwi_palette}, 'NDWI')

folium.LayerControl().add_to(m6)
fig6

In [ ]:
# cell 11 - clip with vector

fig7 = folium.Figure(height="300px")
m7 = folium.Map(location=[18.7883, 98.9853], zoom_start=12).add_to(fig7)

# สร้างฟีเจอร์คอลเลกชันจากพื้นที่ศึกษา (bbox)
study_area_fc = ee.FeatureCollection([ee.Feature(bbox)])
# ตัดภาพด้วยฟีเจอร์คอลเลกชัน (clip)
clipped_img = img.clip(study_area_fc)

# แสดงภาพที่ถูกตัดบนแผนที่
m7.add_raster_layer(clipped_img, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'Clipped Image')

folium.LayerControl().add_to(m7)    
fig7

In [ ]:
# cell 12
# reduce region - คำนวณค่าเฉลี่ย NDVI ในพื้นที่ศึกษา
ndvi_mean = ndvi.reduceRegion(
    reducer=ee.Reducer.mean(),  # ใช้ฟังก์ชัน mean เพื่อคำนวณค่าเฉลี่ย
    geometry=bbox,  # พื้นที่ที่ต้องการคำนวณ
    scale=10,  # ความละเอียดในการคำนวณ (10 เมตรสำหรับ Sentinel-2)
    maxPixels=1e9  # จำนวนพิกเซลสูงสุดที่อนุญาvตในการคำนวณ
)   
print('ค่าเฉลี่ย NDVI ในพื้นที่ศึกษา:', ndvi_mean.getInfo())    


In [ ]:
# cell 13 - time series and matplotlib
import matplotlib.pyplot as plt
import pandas as pd

# ฟังก์ชันคำนวณ NDVI และเพิ่มค่าเฉลี่ยเป็น property ของแต่ละภาพ
def add_ndvi_mean(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    mean = ndvi.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=bbox,
        scale=10,
        maxPixels=1e9
    ).get('NDVI')
    return image.set('ndvi_mean', mean).set('date', image.date().format('YYYY-MM-dd'))

# คำนวณ NDVI เฉลี่ยสำหรับแต่ละภาพใน collection
s2_ndvi = s2.map(add_ndvi_mean)

# ดึงค่า date และ ndvi_mean ออกมาเป็น list
ndvi_list = s2_ndvi.reduceColumns(ee.Reducer.toList(2), ['date', 'ndvi_mean']).get('list').getInfo()

# สร้าง DataFrame และเรียงตามวันที่
df = pd.DataFrame(ndvi_list, columns=['date', 'ndvi_mean'])
df['date'] = pd.to_datetime(df['date'])
df = df.dropna().sort_values('date')

# พล็อตกราฟ NDVI Time Series
plt.figure(figsize=(12, 4))
plt.plot(df['date'], df['ndvi_mean'], marker='o', linestyle='-', color='green', markersize=4)
plt.xlabel('Date')
plt.ylabel('NDVI')
plt.title('NDVI Time Series (Sentinel-2) - Study Area')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
# cell 14 - CHIRPS Rainfall Time Series
# โหลดข้อมูลปริมาณน้ำฝนรายวันจาก CHIRPS (Climate Hazards Group InfraRed Precipitation with Station data)
chirps = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY') \
    .filterDate('2025-01-01', '2025-12-31') \
    .filterBounds(bbox)

print('จำนวนภาพ CHIRPS:', chirps.size().getInfo())

# ฟังก์ชันคำนวณปริมาณน้ำฝนเฉลี่ยในพื้นที่ศึกษา
def add_precip_mean(image):
    mean = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=bbox,
        scale=5566,  # ความละเอียดของ CHIRPS (~0.05 องศา)
        maxPixels=1e9
    ).get('precipitation')
    return image.set('precip_mean', mean).set('date', image.date().format('YYYY-MM-dd'))

# คำนวณปริมาณน้ำฝนเฉลี่ยสำหรับแต่ละภาพ
chirps_mean = chirps.map(add_precip_mean)

# ดึงค่า date และ precip_mean ออกมาเป็น list
precip_list = chirps_mean.reduceColumns(ee.Reducer.toList(2), ['date', 'precip_mean']).get('list').getInfo()

# สร้าง DataFrame และเรียงตามวันที่
df_rain = pd.DataFrame(precip_list, columns=['date', 'precipitation'])
df_rain['date'] = pd.to_datetime(df_rain['date'])
df_rain = df_rain.dropna().sort_values('date')

# พล็อตกราฟ NDVI และ Rainfall รวมกัน (dual axis)
fig, ax1 = plt.subplots(figsize=(12, 5))

# แกนซ้าย - ปริมาณน้ำฝน (bar chart)
ax1.bar(df_rain['date'], df_rain['precipitation'], color='steelblue', alpha=0.6, label='Rainfall (mm/day)')
ax1.set_xlabel('Date')
ax1.set_ylabel('Precipitation (mm/day)', color='steelblue')
ax1.tick_params(axis='y', labelcolor='steelblue')

# แกนขวา - NDVI (line chart)
ax2 = ax1.twinx()
ax2.plot(df['date'], df['ndvi_mean'], marker='o', linestyle='-', color='green', markersize=4, label='NDVI')
ax2.set_ylabel('NDVI', color='green')
ax2.tick_params(axis='y', labelcolor='green')

plt.title('NDVI & CHIRPS Rainfall Time Series (2025) - Study Area')
fig.legend(loc='upper left', bbox_to_anchor=(0.12, 0.95))
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
# cell 13 - export image to drive
# กำหนดพารามิเตอร์การส่งออกภาพ
export_params = {
    'scale': 10,  # ความละเอียดของภาพ (10 เมตรสำหรับ Sentinel-2)
    'region': bbox,  # พื้นที่ที่จะส่งออก (ใช้ bbox     ที่สร้างไว้ก่อนหน้านี้)
    'fileFormat': 'GeoTIFF', 
    'folder': 'GEE_Exports',  # โฟลเดอร์ใน Google Drive ที่จะเก็บไฟล์
    'fileNamePrefix': 'sentinel2_image',  # ชื่อไฟล์ที่จะส่งออก (จะมีการเติมวันที่และเวลาอัตโนมัติ)
    'maxPixels': 1e9  # กำหนดจำนวนพิกเซลสูงสุดที่อนุญาตให้ส่งออก (ปรับตามขนาดภาพ)
}   

# การส่งออกภาพไปยัง Google Drive
task = ee.batch.Export.image.toDrive(image=img, description='Export_Sentinel2_Image', **export_params)
task.start()    


